In [1]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import os
from dotenv import load_dotenv
import psycopg2

In [2]:
load_dotenv()

# Access environment variables
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")

In [3]:
df = pd.read_csv('marks.csv')
df.head()

,student_class_id,marks
0,1,10
1,2,20
2,3,32
3,4,40
4,9,50


In [4]:
out_of = 200
week = 1
min_marks = 0
max_marks = 100

In [5]:
df['marks'] = (df['marks'] / out_of) * 100
df.head()

,student_class_id,marks
0,1,5.0
1,2,10.0
2,3,16.0
3,4,20.0
4,9,25.0


In [6]:
X = df.drop("student_class_id", axis=1)

gmm = GaussianMixture(n_components=5, random_state=0)
df['tier'] = gmm.fit_predict(X)
df.head()

,student_class_id,marks,tier
0,1,5.0,3
1,2,10.0,3
2,3,16.0,0
3,4,20.0,0
4,9,25.0,4


In [7]:
tier_df = df[["student_class_id", "tier"]]

# Show DataFrame
tier_df

,student_class_id,tier
0,1,3
1,2,3
2,3,0
3,4,0
4,9,4
5,121,4
6,122,1
7,123,1
8,124,2
9,125,2


In [8]:
import psycopg2
from datetime import datetime

conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)



def update_institute_class_students(conn, update_data):
    cur = conn.cursor()

    for index, row in update_data.iterrows():
        student_id = int(row['student_class_id']) 
        tier = int(row['tier'])  

        update_query = """
        UPDATE institute_class_students
        SET tier = %s, tier_last_updated_at = %s
        WHERE student_id = %s
        """
        
        current_time = datetime.now()
        # Execute the update query with the parameters
        cur.execute(update_query, (tier, current_time, student_id))

    # Commit the changes to the database
    conn.commit()
    cur.close()

update_institute_class_students(conn, tier_df)

In [9]:
def insert_into_institute_class_tier_students(conn, insert_data):
    cur = conn.cursor()

    for index, row in insert_data.iterrows():
        student_id = int(row['student_class_id']) 
        tier = int(row['tier'])
        marks = float(row['marks'])  # Added 'marks' for insertion into relevant table.

        # First, we select the class and institute information for the student.
        select_query = """
        SELECT class_id, institute_id 
        FROM institute_class_students 
        WHERE id = %s
        """
        cur.execute(select_query, (student_id,))
        result = cur.fetchone()
        
        # If there's no matching class or institute for this student_id, continue to the next row
        if not result:
            continue
        
        class_id, institute_id = result

        # Insert data into 'institute_class_tiers'
        insert_tier_query = """
        INSERT INTO institute_class_tiers (class_id, institute_id, created_at, min_mark, max_mark, week, updated_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        RETURNING id
        """
        current_time = datetime.now()
        min_marks, max_marks, week = 0, 100, 1  # Example values, adjust as needed.
        cur.execute(insert_tier_query, (class_id, institute_id, current_time, min_marks, max_marks, week, current_time))
        tier_id = cur.fetchone()[0]

        # Insert data into 'institute_class_tier_students'
        insert_student_query = """
        INSERT INTO institute_class_tier_students (class_id, institute_id, tier_id, student_class_id, created_at, tier, marks)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        cur.execute(insert_student_query, (class_id, institute_id, tier_id, student_id, current_time, tier, marks))

    # Commit the changes to the database
    conn.commit()
    cur.close()

insert_into_institute_class_tier_students(conn, df)

ForeignKeyViolation: insert or update on table "institute_class_tier_students" violates foreign key constraint "fk_institute_tier_id"
DETAIL:  Key (class_id)=(1) is not present in table "institute_class_tiers".
